## Import

In [1]:
import os

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/eiroamd/Documents/IH/Ironhack Data Analytics.json"

In [3]:
from google.cloud import bigquery

In [4]:
client = bigquery.Client()

## Challenge 1

### Step 1

In [5]:
query1 = """
SELECT
    a.au_id,
    t_a.title_id,
    ((t.price * s.qty)*(t.royalty/100)*(t_a.royaltyper/100)) AS sales_royalty
    
FROM
    `ironhack-data-analytics.publications.authors` a
INNER JOIN
    `ironhack-data-analytics.publications.titleauthor` t_a
USING 
    (au_id)

LEFT JOIN
    `ironhack-data-analytics.publications.titles` t
USING
    (title_id)

INNER JOIN
    `ironhack-data-analytics.publications.sales` s
USING
    (title_id)

GROUP BY
    s.qty,
    t.royalty,
    t_a.royaltyper,
    t.price,
    a.au_id,
    t_a.title_id
    
ORDER BY
    t_a.title_id
"""

In [6]:
query_job = client.query(query=query1)
df = query_job.to_dataframe()
df.head(20)

,au_id,title_id,sales_royalty
0,213-46-8915,BU1032,3.998
1,213-46-8915,BU1032,7.996
2,409-56-7008,BU1032,5.997
3,409-56-7008,BU1032,11.994
4,724-80-9391,BU1111,17.925
5,267-41-2394,BU1111,11.950
6,213-46-8915,BU2075,25.116
7,274-80-9391,BU7832,29.985
8,712-45-1867,MC2222,23.988
9,899-46-2035,MC3021,4.485


### Step 2

In [7]:
query2 = """

SELECT
    title_id,
    au_id,
    SUM(sales_royalty) AS agg_royalties

FROM
(SELECT
    a.au_id,
    t_a.title_id,
    ((t.price * s.qty)*(t.royalty/100)*(t_a.royaltyper/100)) AS sales_royalty

FROM
    `ironhack-data-analytics.publications.authors` a
INNER JOIN
    `ironhack-data-analytics.publications.titleauthor` t_a
USING 
    (au_id)

LEFT JOIN
    `ironhack-data-analytics.publications.titles` t
USING
    (title_id)

INNER JOIN
    `ironhack-data-analytics.publications.sales` s
USING
    (title_id)

GROUP BY
    s.qty,
    t.royalty,
    t_a.royaltyper,
    t.price,
    a.au_id,
    t_a.title_id
    
ORDER BY
    t_a.title_id) royalties

GROUP BY
    au_id,
    title_id
ORDER BY
    title_id
"""

In [8]:
query_job = client.query(query=query2)
df = query_job.to_dataframe()
df.head(20)

,title_id,au_id,agg_royalties
0,BU1032,213-46-8915,11.994
1,BU1032,409-56-7008,17.991
2,BU1111,267-41-2394,11.950
3,BU1111,724-80-9391,17.925
4,BU2075,213-46-8915,25.116
5,BU7832,274-80-9391,29.985
6,MC2222,712-45-1867,23.988
7,MC3021,722-51-5454,21.528
8,MC3021,899-46-2035,7.176
9,PC1035,238-95-7766,110.160


### Step 3

In [9]:
query3 = """
SELECT
    au_id,
    advance + agg_royalties AS total_profit

FROM
(SELECT
    title_id,
    au_id,
    advance,
    SUM(sales_royalty) as agg_royalties

FROM
(SELECT
    a.au_id,
    t_a.title_id,
    t.advance,
    ((t.price * s.qty)*(t.royalty/100)*(t_a.royaltyper/100)) AS sales_royalty

FROM
    `ironhack-data-analytics.publications.authors` a
INNER JOIN
    `ironhack-data-analytics.publications.titleauthor` t_a
USING 
    (au_id)

LEFT JOIN
    `ironhack-data-analytics.publications.titles` t
USING
    (title_id)

INNER JOIN
    `ironhack-data-analytics.publications.sales` s
USING
    (title_id)

GROUP BY
    s.qty,
    t.royalty,
    t_a.royaltyper,
    t.price,
    a.au_id,
    t_a.title_id,
    t.advance
    
ORDER BY
    t_a.title_id) royalties

GROUP BY
    au_id,
    title_id,
    advance
    
ORDER BY
    title_id) aggregated_royalties

GROUP BY
    au_id,
    advance,
    agg_royalties

ORDER BY
    total_profit DESC
LIMIT
    3
"""

In [10]:
query_job = client.query(query=query3)
df = query_job.to_dataframe()
df.head(20)

,au_id,total_profit
0,722-51-5454,15021.528
1,899-46-2035,15007.176
2,213-46-8915,10150.116


## Challenge 2

In [22]:
query4 = """
CREATE TABLE `ironhack-data-analytics.publications.most_profiting_authors_dani`
AS( SELECT 
    au_id,
    advance + agg_royalties AS profits

FROM
(SELECT
    title_id,
    au_id,
    advance,
    SUM(sales_royalty) as agg_royalties

FROM
(SELECT
    a.au_id,
    t_a.title_id,
    t.advance,
    ((t.price * s.qty)*(t.royalty/100)*(t_a.royaltyper/100)) AS sales_royalty

FROM
    `ironhack-data-analytics.publications.authors` a
INNER JOIN
    `ironhack-data-analytics.publications.titleauthor` t_a
USING 
    (au_id)

LEFT JOIN
    `ironhack-data-analytics.publications.titles` t
USING
    (title_id)

INNER JOIN
    `ironhack-data-analytics.publications.sales` s
USING
    (title_id)

GROUP BY
    s.qty,
    t.royalty,
    t_a.royaltyper,
    t.price,
    a.au_id,
    t_a.title_id,
    t.advance
    
ORDER BY
    t_a.title_id) royalties

GROUP BY
    au_id,
    title_id,
    advance
    
ORDER BY
    title_id) aggregated_royalties

GROUP BY
    au_id,
    advance,
    agg_royalties)
"""

In [23]:
query_job = client.query(query=query4)
df = query_job.to_dataframe()
df.head(20)

Conflict: 409 GET https://www.googleapis.com/bigquery/v2/projects/ironhack-data-analytics/queries/6ad1331b-0613-44e1-8cb6-8d76e72554a5?maxResults=0&location=US: Already Exists: Table ironhack-data-analytics:publications.most_profiting_authors_dani

In [26]:
query5 = """
SELECT *
FROM
    `ironhack-data-analytics.publications.most_profiting_authors_dani`
ORDER BY
    profits DESC
"""

In [27]:
query_job = client.query(query=query5)
df = query_job.to_dataframe()
df.head(20)

,au_id,profits
0,899-46-2035,15007.176
1,472-27-2349,8008.994
2,807-91-6654,7083.800
